In [ ]:
import pandas as pd 
import numpy as np
import datetime

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates
mpl.rcParams['font.family'] = 'SF Compact Text'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
df = pd.read_csv('./fhv_tripdata_2018-01.csv')

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df['Pickup_DateTime'] = pd.to_datetime(df['Pickup_DateTime'])

df = df.set_index('Pickup_DateTime', drop = True)

In [ ]:
df.head(5)

In [ ]:
df['Trips'] = ""

df = df['Trips']

In [ ]:
df = df.resample('M').count()

df = pd.DataFrame(df)

In [ ]:
df.head(5)

In [ ]:
weather = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in ['2018-01-04', '2018-03-21','2017-03-14','2017-02-09','2016-01-23']]
# holidays = [date for y in range(2015, 2019) for date, _ in sorted(holidays.US(years=y).items())]
holidays = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in ['2018-01-07', '2018-03-21','2017-03-14','2017-02-09','2016-01-23']]

df['is_holiday'], df['bad_weather'] = 0, 0

df['is_holiday'][df.index.isin(holidays)] = 1
df['bad_weather'][df.index.isin(weather)] = 1

# df['bad_weather'] = np.where(df.index.isin(weather), 1, 0)
df['next_is_holiday'] = df.is_holiday.shift(-1)
df['next_bad_weather'] = df.bad_weather.shift(-1)

In [ ]:
def create_dataset(dataset, look_back=1, forecast_horizon=1, batch_size=1):
    batch_x, batch_y, batch_z = [], [], []
    for i in range(0, len(dataset)-look_back-forecast_horizon-batch_size+1, batch_size):
        for n in range(batch_size):            
            x = dataset[['log_counts','next_is_holiday','next_bad_weather']].values[i+n:(i + n + look_back), :]
            offset = x[0, 0]
            y = dataset['log_counts'].values[i + n + look_back:i + n + look_back + forecast_horizon]
            batch_x.append(np.array(x).reshape(look_back, -1))
            batch_y.append(np.array(y))
            batch_z.append(np.array(offset))
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        batch_z = np.array(batch_z)
        batch_x[:, :, 0] -= batch_z.reshape(-1, 1)
        batch_y -= batch_z.reshape(-1, 1)
        yield batch_x, batch_y, batch_z
        batch_x, batch_y, batch_z = [], [], []

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        self.hidden_size = 128
        self.bi = 1
        self.lstm = nn.LSTM(config.get('features'),self.hidden_size,1,dropout=0.1,bidirectional=self.bi-1,batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size,self.hidden_size // 4,1,dropout=0.1,bidirectional=self.bi-1,batch_first=True)
        self.dense = nn.Linear(self.hidden_size // 4, config.get('forecast_horizon'))
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x, batch_size=100):
        hidden = self.init_hidden(batch_size)
        output, _ = self.lstm(x, hidden)
        output = F.dropout(output, p=0.5, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=0.5, training=True)
        output = self.dense(state[0].squeeze(0))
        
        return output
        
    def init_hidden(self, batch_size):
        h0 = Variable(torch.zeros(self.bi, batch_size, self.hidden_size))
        c0 = Variable(torch.zeros(self.bi, batch_size, self.hidden_size))
        return h0, c0
    
    def init_hidden2(self, batch_size):
        h0 = Variable(torch.zeros(self.bi, batch_size, self.hidden_size//4))
        c0 = Variable(torch.zeros(self.bi, batch_size, self.hidden_size//4))
        return h0, c0
    
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

In [ ]:
batch_size = 1
forecast_horizon = 1
look_back = 28
model = Model(dict(features=3, forecast_horizon=1))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
n_epochs = 5

model.train()
train_true_y = []
train_pred_y = []
for epoch in range(n_epochs):
    ep_loss = []
    for i, batch in enumerate(create_dataset(df[df.index<"2018-09-01"], look_back=look_back, forecast_horizon=1, batch_size=batch_size)):
        print("[{}{}] Epoch {}: loss={:0.4f}".format("-"*(20*i//(len(df[df.index<"2018"])//batch_size)), " "*(20-(20*i//(len(df[df.index<"2018"])//batch_size))),epoch, np.mean(ep_loss)), end="\r")
        try:
            batch = [torch.Tensor(x) for x in batch]
        except:
            break

        out = model.forward(batch[0].float(), batch_size)
        loss = model.loss(out, batch[1].float())
        if epoch == n_epochs - 1:
            train_true_y.append((batch[1] + batch[2]).detach().numpy().reshape(-1))
            train_pred_y.append((out + batch[2]).detach().numpy().reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ep_loss.append(loss.item())
    print()